In [36]:
import cv2
import os
import shutil
import imghdr
import numpy as np
import matplotlib.pyplot as plt

In [88]:
# 方案一：百分比阈值法
# 原理：取图像中最亮的前N%像素作为高光区域
# percentile=95 表示取最亮的前5%像素作为高光
# 适用于大多数常规场景
# 可通过调整percentile参数控制敏感度
def create_mask(imgpath, percentile=98):
    image = cv2.imread(imgpath, cv2.IMREAD_GRAYSCALE)
    """通过亮度百分比确定阈值"""
    # 计算指定百分位的像素值
    threshold = np.percentile(image, percentile)
    _, mask = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
    return mask

In [89]:
def xiufu(imgpath, maskpath):
    src = cv2.imread(imgpath)
    mask = cv2.imread(maskpath, cv2.IMREAD_GRAYSCALE)
    res = cv2.resize(src, None, fx=0.6, fy=0.6, 
                     interpolation=cv2.INTER_CUBIC)
    mask = cv2.resize(mask, None, fx=0.6, fy=0.6, 
                      interpolation=cv2.INTER_CUBIC)
    dst = cv2.inpaint(res, mask, 10, cv2.INPAINT_NS)
    return dst

In [90]:
def process_images(rootpath, masksavepath, savepath):
    imgfiles = [f for f in os.listdir(rootpath) if imghdr.what(os.path.join(rootpath, f)) in ['jpeg', 'png', 'gif', 'bmp']]
    
    for imgfile in imgfiles:
        imgpath = os.path.join(rootpath, imgfile)
#         print(imgfile)

        maskpath = os.path.join(masksavepath, "mask_" + imgfile)
        cv2.imwrite(maskpath, create_mask(imgpath))

        dst = xiufu(imgpath, maskpath)
        newname = imgfile.split(".")[0]
        dstresize = cv2.resize(dst,(384,288))
        cv2.imwrite(os.path.join(savepath, newname + ".jpg"), dstresize)

In [91]:

if __name__ == "__main__":
    rootpath = r"F:\wujieyu\img test\TestDataset\Kvasir\images\img"
    masksavepath=r"F:\wujieyu\img test\TestDataset\Kvasir\img Highlights processing NS\Highlights processing percentile=98\highlightmask"
    savepath = r"F:\wujieyu\img test\TestDataset\Kvasir\img Highlights processing NS\Highlights processing percentile=98\result"

    for directory in [rootpath, masksavepath, savepath]:
        os.makedirs(directory, exist_ok=True)

    process_images(rootpath, masksavepath, savepath)
    print("处理完成")

处理完成
